In [7]:
import os, sys
import mujoco
import mujoco.viewer
import numpy as np
import time
import torch

import sys
sys.path.append("F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL")  # ← Adjust this
from learning.model import FeatureAttentionStatePredictor


## Visualization of original trajactory

The following block allows you to view the trajectory from MPPI

In [13]:
# Load model to get nq and nv
model = mujoco.MjModel.from_xml_path("F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\Visualization\\humanoid.xml")
nq = model.nq
nv = model.nv

# Load your state file
data_directory = "F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\data\\2025-04-19_153833"
state_path = os.path.join(data_directory, "states.csv")
states = np.loadtxt(state_path, delimiter=",", skiprows=1)  # adjust if header
print(f"Loaded {states.shape[0]} frames, {states.shape[1]} features per state.")

# Validate and slice if necessary
assert states.shape[1] >= nq + nv, "State file doesn't contain enough dimensions for qpos + qvel"

qpos_qvel = states[:, :nq+nv]  # Take only the first nq+nv columns

# Save as trajectory.csv
np.savetxt("trajectory.csv", qpos_qvel, delimiter=",")
print("Saved trajectory.csv!")

# Load the model
data = mujoco.MjData(model)

# Dummy predicted trajectory (replace with your real output)
# Shape: (T, qpos_dim + qvel_dim)
trajectory = np.loadtxt("trajectory.csv", delimiter=",")  # ← or from your model

# Launch viewer
with mujoco.viewer.launch_passive(model, data) as viewer:
    for t in range(len(trajectory)):
        # Unpack qpos and qvel
        qpos_dim = model.nq
        qvel_dim = model.nv
        qpos = trajectory[t, :qpos_dim]
        qvel = trajectory[t, qpos_dim:qpos_dim+qvel_dim]

        # Set state
        data.qpos[:] = qpos
        data.qvel[:] = qvel

        # Forward and render
        mujoco.mj_forward(model, data)
        viewer.sync()

        time.sleep(1 / 60.0)  # 60 FPS

Loaded 24031 frames, 55 features per state.
Saved trajectory.csv!


KeyboardInterrupt: 

## Visualization of State-predictor Trajectory

The following block allows you to view the predicted state trajectory

In [8]:
# Define model parameters
state_dim = 55  # Replace with your actual state dimension
action_dim = 21  # Replace with your actual action dimension

# Initialize the model 

# cartpole: state_dim=4, action_dim=1, hidden_dim=64, num_heads=4, attn_layers=2
# humanoid: state_dim=55, action_dim=21, hidden_dim=256, num_heads=4, attn_layers=4
model = FeatureAttentionStatePredictor(
    state_dim=4,
    action_dim=1,
    hidden_dim=64,
    num_heads=4,
    attn_layers=2,
    dropout_rate=0.0
)

# Load the trained weights
pth_path = "F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\checkpoints_cartpole\\model_final.pth"
model.load_state_dict(torch.load(pth_path, map_location='cpu'))
model.eval()

FeatureAttentionStatePredictor(
  (feature_encoding): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
  )
  (layers): ModuleList(
    (0-1): 2 x ModuleDict(
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
      )
      (dropout1): Dropout(p=0.0, inplace=False)
      (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ffn): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.0, inplace=False)
        (3): Linear(in_features=256, out_features=64, bias=True)
      )
      (dropout2): Dropout(p=0.0, inplace=False)
    )
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
# Load initial states and actions
# 04-21_011138 is for cartpole inferencing, 04-19_153833 is for humanoid
data_directory = "F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\data\\2025-04-21_011138"
state_path = os.path.join(data_directory, "states.csv")
action_path = os.path.join(data_directory, "actions.csv")
states = np.loadtxt(state_path, delimiter=',')  # Shape: (T, state_dim)
actions = np.loadtxt(action_path, delimiter=',')  # Shape: (T, action_dim)

# Initialize a list to store predicted states
predicted_states = []

# Convert to tensors
state_tensor = torch.tensor(states[0], dtype=torch.float32)
for action in actions:
    action_tensor = torch.tensor(action, dtype=torch.float32)
    if action_tensor.ndim == 0:
        action_tensor = action_tensor.unsqueeze(0)  # Make it 1D
    input_tensor = torch.cat((state_tensor, action_tensor)).unsqueeze(0)  # Shape: (1, state_dim + action_dim)
    
    with torch.no_grad():
        delta = model(input_tensor).squeeze(0)  # Predict the change in state
    
    next_state = state_tensor + delta  # Update the state
    predicted_states.append(next_state.numpy())
    state_tensor = next_state  # Set the current state for the next iteration

# Convert the list to a NumPy array
predicted_states = np.array(predicted_states)
np.savetxt('trajectory.csv', predicted_states, delimiter=',')


In [15]:
# Load the MuJoCo model
# model_path = './humanoid.xml'  # Replace with your actual model path
model_path = "F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\models\\cartpole.xml"
mujoco_model = mujoco.MjModel.from_xml_path(model_path)
mujoco_data = mujoco.MjData(mujoco_model)

# Load the predicted trajectory
trajectory = np.loadtxt('trajectory.csv', delimiter=',')

# Determine dimensions
qpos_dim = mujoco_model.nq
qvel_dim = mujoco_model.nv

# Launch the viewer
with mujoco.viewer.launch_passive(mujoco_model, mujoco_data) as viewer:
    for state in trajectory:
        # Split the state into qpos and qvel
        qpos = state[:qpos_dim]
        qvel = state[qpos_dim:qpos_dim + qvel_dim]

        # Set the state
        mujoco_data.qpos[:] = qpos
        mujoco_data.qvel[:] = qvel

        # Forward the simulation and render
        mujoco.mj_forward(mujoco_model, mujoco_data)
        viewer.sync()

        time.sleep(1 / 60.0)  # Adjust the sleep time as needed
